In [2]:

# here put the import lib
import os

import logging as log
import pandas as pd

from modules.logger.logger import init as logInit
from modules.logUtils.httpEvent import HttpEvent, TABLE_LABELS
from modules.fileUtils.fileUtils import getFilesByPath
from modules.config.argumentParser import argumentParser
from modules.config.configParser import getValue as getValueFromConfig

# CONFIG

TABLE_LABELS = [
    "timestamp",
    "threadId",
    "institutionId",
    "userId",
    "url",
    "method",
    "statusCode",
    "parameterType",
    "parameterName",
    "parameterValue",
    "header",
    "ip",
    "port",
    "vpnip"
]
# get all files.
files = getFilesByPath("data/origin/syjs")
if not files: 
    print("there no any file. recorrect the input path.")
files = files[:200]

print(len(files))
print(files)

# IF SMIPLY WRITE TO CSV
# with open(os.path.join(output, "", "tmp.csv"), "w", encoding="utf-8") as target:
#     target.write(TABLE_LABELS+'\n')
# END


200
['data/origin/syjs/fa/api.log', 'data/origin/syjs/fa/api.log_2020-09-28.log', 'data/origin/syjs/fa/api.log_2020-10-14.log', 'data/origin/syjs/fa/icaslog.log', 'data/origin/syjs/fa/icaslog.log_2020-09-20.log', 'data/origin/syjs/fa/icaslog.log_2020-09-21.log', 'data/origin/syjs/fa/icaslog.log_2020-09-22.log', 'data/origin/syjs/fa/icaslog.log_2020-09-23.log', 'data/origin/syjs/fa/icaslog.log_2020-09-24.log', 'data/origin/syjs/fa/icaslog.log_2020-09-25.log', 'data/origin/syjs/fa/icaslog.log_2020-09-26.log', 'data/origin/syjs/fa/icaslog.log_2020-09-27.log', 'data/origin/syjs/fa/icaslog.log_2020-09-28.log', 'data/origin/syjs/fa/icaslog.log_2020-09-29.log', 'data/origin/syjs/fa/icaslog.log_2020-09-30.log', 'data/origin/syjs/fa/icaslog.log_2020-10-01.log', 'data/origin/syjs/fa/icaslog.log_2020-10-02.log', 'data/origin/syjs/fa/icaslog.log_2020-10-03.log', 'data/origin/syjs/fa/icaslog.log_2020-10-04.log', 'data/origin/syjs/fa/icaslog.log_2020-10-05.log', 'data/origin/syjs/fa/icaslog.log_2020

In [3]:

data = []
for file in files:
    count = 0
    with open(file, 'r', encoding='utf-8') as f:
        # read raw log by line.
        for line in f:
            try:
                he = HttpEvent(line)
                count += 1

                # filter url
                subs = he.url.split('/')
                for i, sub in enumerate(subs):
                    if sub == "": continue
                    if sub == he.userId: subs[i] = "userId"
                    elif sub == he.institutionId: subs[i] = "institutionId"
                    elif sub == he.name: subs[i] = "name"
                    elif len(sub) == 32: subs[i] = "hash"
                    elif i > 1:
                        if 'By' in subs[i-1]:
                            subs[i] = subs[i-1].split('By')[-1]
                        elif 'Code' in subs[i-1]:
                            subs[i] = 'code'
                        elif subs[i-1] == 'assetTag':
                            subs[i] = 'tags'
                            subs = subs[:i+1]
                            break
                        elif subs[i-1].endswith('UserLeadrName'):
                            subs[i] = 'userId'
                        elif subs[i-1].startswith('get') or subs[i-1].startswith('Get'):
                            if subs[i-1].endswith('Info'):
                                subs[i] = 'info'
                            elif subs[i-1].endswith('Code') or subs[i-1].endswith('Codes'):
                                subs[i] = 'code' 
                he.url = "/".join(subs)

                # append to Data
                data.extend([he.getSet()])
            except Exception as e:
                log.debug(e)
    log.info("{0} write {1} lines".format(file, count))
print(len(data), len(data[0]))
# convert List to DataFrame
# print(data[0])
df = pd.DataFrame(data = data, columns=TABLE_LABELS)


181328 14


In [12]:
)

SyntaxError: invalid syntax (<ipython-input-12-5e9d5648d334>, line 2)

In [4]:
urlDF = df.urls
for k, v in urls:
    print(k, v)

NameError: name 'urls' is not defined

In [83]:
import collections
dic = collections.defaultdict(list)

for url in subUrls:
    dic[len(url)].append(url)
for i, j in dic.items():
    print(i, j)
# 按照长度进行统计

7 ['accbook', 'inasset', 'prjInfo']
42 ['remoteGetDepreciateExeBudgetPageForSetting', 'remoteGetAssetExtendInfoListByAssetIdsPost']
8 ['userInfo', 'wordFlow', 'm=delete', 'purchase', 'assetAcc', 'deptInfo', 'assetTag', 'supplier']
31 ['remoteCheckIsCanCloseCurrPeriod', 'remoteGetPagedInAssetModifyList', 'getExeBudgetInfoListByBudgetIds']
35 ['remoteGetPagedAssetListByPermission']
41 ['remoteGetPagedAssetListByUserOrPermission']
12 ['assetReceive', 'downloadFile', 'bizWfMapping']
27 ['remoteGetParamConfigByCodes', 'getInAssetSendVoucherResult']
19 ['remoteGetAllAccbook', 'accAssetVoucherDepr', 'getDepreciateResult', 'submitApproveOption', 'remoteGetIsEnableOU']
6 ['config', 'userId', 'period', 'DeptId', 'ledger']
11 ['inAssetRoom', 'assetPrjMap', 'sendVoucher', 'paramConfig', 'AssetModify', 'assetModify', 'voucherInfo', 'queryByCode', 'stockInCode', 'InAssetInfo', 'inAssetInfo']
13 ['InAssetModify', 'inAssetModify', 'bgUsageBudget', 'codeGenerator']
24 ['batchUpdateImportBizData', 'getE

In [85]:
def convertUrl(url: str)->str:
    subs = url.split('/')
    subs = subs[1:] if subs[0] == "" else subs
    # print(subs)
    for i, sub in enumerate(subs):
        if len(sub) == 32:
            subs[i] = "id"
        elif i > 1:
            if "By" in sub:
                subs[i] = sub[i-1].split('By')[-1]
            
            # if "ByCode" in subs[i-1] or "ByCodes" in subs[i-1] or "InCode" in subs[i-1]:
            #     subs[i] = "code"
            # elif "ById" in subs[i-1] or "ByIds" in subs[i-1]:
            #     subs[i] = "id"
    
    return "/" + "/".join(subs)
                




dic = dict()
for url in urls:
    template = convertUrl(url)
    dic[template] = 1 if template not in dic else dic[template] + 1

for k,v in dic.items():
    print(v, k)



1 /fa/asset/assetInfo/t/Id
1 /fa/commoncore/userInfo/remoteGetCurrenUserLeadrName/321121-120
1 /fa/commoncore/uniqueValidate/uniqueValidate/AssetInfo/assetTag/tags
1 /fa/ledger/accbook/updateCurrPeriod/hash
1 /fa/purchase/assetAcpt
1 /fa/ledger/inAssetDepreciate/getInassetDepreciateResult
1 /fa/ledger/voucherInfo/remoteGetPagedVoucherInfoQueryList
1 /fa/asset/assetModify/batchUpdateImportBizData
1 /fa/ledger/accAssetVoucherDepr
1 /fa/config/attachment/t
1 /fa/commoncore/assetCommonCore/getAssetClasss
1 /fa/inasset/inAssetModify/batchImportBizData
1 /fa/commoncore/bizWfMapping/t
1 /fa/ledger/assetDepreciateRecord/assetDepreciateRecord
1 /fa/asset/assetReceive/getPagedMyAssetApply
1 /fa/ledger/accbookPermission
1 /fa/ledger/period/remoteGetAllPeriod
1 /fa/config/externalConfig/e/Code
1 /fa/commoncore/prjInfo/getOpenedPrjInfoList
1 /fa/ledger/inAssetDepreciate
1 /fa/ledger/accbook/t
1 /fa/commoncore/assetCommonCore/remoteGetCurrenUserInfo
1 /fa/commoncore/wordFlow/remoteGetWorkFlowIsEnd
1